In [6]:
# Script/ingest.py
# Create a chroma db vectors
import json
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings

embedder = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2") # Small, fast, local

docs = []
metadatas = []

with open("data/aircon_troubleshooting_sharegpt_100.jsonl") as f:
    for line in f:
        obj = json.loads(line)
        q = obj["conversations"][0]["value"]
        a = obj["conversations"][1]["value"]
        docs.append(q)
        metadatas.append({"answer": a})

C:\Users\Toto\AppData\Local\Temp\ipykernel_4920\1466034751.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2") # Small, fast, local
e:\RAG_About_Me\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [ ]:
docs[:5]

['My Carrier window-type air conditioner remote shows error code U4. What should I do?',
 'My Carrier window-type air conditioner shows error code E1. What should I do?',
 'My Carrier portable air conditioner is running but not cooling. What should I do?',
 'My Daikin non-inverter air conditioner makes a clicking sound. What should I do?',
 'My Panasonic portable air conditioner produces a burning smell. What should I do?']

In [ ]:
metadatas[:5] 

[{'answer': 'This error usually means a communication problem between indoor and outdoor units. Try resetting power to both units. If unresolved, consult a service center.'},
 {'answer': 'Error code E1 often means a sensor fault. Try resetting the AC. If the error stays, have the sensor checked by a professional.'},
 {'answer': 'First, make sure the air filter is clean and not clogged. Also, confirm the unit is set to cooling mode, and check if the thermostat is set below room temperature. If these are fine, low refrigerant or a faulty compressor may be the issue, and a technician is needed.'},
 {'answer': 'A clicking sound often indicates a relay issue or loose fan. Turn off the AC and inspect for loose parts, or call a technician.'},
 {'answer': 'A burning smell can indicate electrical issues or motor failure. Turn off the unit immediately and call for service.'}]

In [ ]:
# ignore the error its safe
db = Chroma.from_texts(
    docs, 
    embedder, 
    persist_directory="./chroma_db", 
    metadatas=metadatas
)

# db.persist() # this is atutomatic in chroma new version
print("Ingest complete. ChromaDB created.")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


----

In [ ]:
# backend/retrieval
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
import httpx

SIM_THRESHOLD = 0.7  # Lower for small dataset, adjust as needed

embedder = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma(
    persist_directory="./chroma_db", 
    embedding_function=embedder
)

C:\Users\Toto\AppData\Local\Temp\ipykernel_11548\1005270666.py:9: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
def retrieve(query):
    docs = db.similarity_search_with_score(query, k=1)
    if not docs or docs[0][1] < SIM_THRESHOLD:
        return None
    doc = docs[0][0]
    return doc.page_content, doc.metadata["answer"]

In [ ]:
async def call_ollama(prompt):
    url = "http://ollama:11434/api/generate"
    payload = {
        "model": 'gemma3n:e4b-it-q4_K_M',
        "prompt": prompt,
        "stream": False
    }
    async with httpx.AsyncClient() as client:
        resp = await client.post(url, json=payload)
        result = resp.json()
        return result["response"].strip()

---

In [ ]:
# About me

In [ ]:
# Load your PDF with LangChain’s PDFLoader:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/About_Me.pdf")
pages = loader.load()


In [10]:
# Split the text into chunks (for embedding):
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Start with 500/100 or 1000/200, then adjust if you get incomplete or duplicated answers.
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = splitter.split_documents(pages)


In [43]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import httpx

SIM_THRESHOLD = 0.9  # Lower for small dataset, adjust as needed

embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma(
    persist_directory="./chroma_db", 
    embedding_function=embedder)



 

In [47]:
query = 'foods?'

In [48]:
docs = db.similarity_search_with_score(query, k=1)
SIM_THRESHOLD = 0.1

In [49]:
if not docs or docs[0][1] > SIM_THRESHOLD:
    # No good match found
    print("No similar enough document found.")
else:
    doc = docs[0][0]
    print(doc.page_content, doc.metadata["answer"])


No similar enough document found.
